# Time Series Data Example: Predicting Entity Class Performance

This notebook helps you solve a machine learning task based on a synthetic time series data.

## Problem objective

Given a dataset where:
- Each **row** represents the target values for a specific entity, value kind, and time step,
- Entities belong to specific classes, have properties, and sizes at any given time.
- Two targets (`target_1` and `target_2`) exist.

Your goal is to **identify the entity class** that, on average, has the **lowest ratio** of:

$$
\text{ratio} = \frac{\text{target}_1}{\text{target}_2}
$$

You will use ML models to answer this question.

## The data

In [ ]:
# Required Libraries
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer

import os
from google.colab import drive

**Note:** This notebook assumes the dataset is stored on **Google Drive**, for me, this is the location the file is at:

`/content/drive/MyDrive/Colab Notebooks/CMPT 3510 W25/Project/`

Download the dataset from [here](https://drive.google.com/file/d/12s_av600d3DG0j3nGzTGV0Ea5w5CZqu4/view?usp=drive_link). Please **update the file path** below to match the location where **you have saved the dataset** in your own Google Drive or local environment.

In [ ]:
# Mount Google Drive
drive.mount("/content/drive")

base_path = "/content/drive/MyDrive/Colab Notebooks/CMPT 3510 W25/Project/"

Mounted at /content/drive


In [ ]:
# Load the data
csv_file = os.path.join(base_path, "synthetic_targets.csv")
df = pd.read_csv(csv_file)
df

,time_index,entity_name,entity_class,property_class,size,value_kind,target_1,target_2
0,1,Entity_198,Class_4,Prop_1,590.0,B,1038.0,386.9
1,1,Entity_198,Class_4,Prop_1,590.0,C,605.1,1315.0
2,1,Entity_198,Class_4,Prop_1,590.0,F,626.0,1655.0
3,1,Entity_198,Class_4,Prop_1,590.0,G,570.7,1616.0
4,1,Entity_198,Class_4,Prop_1,590.0,Q,1291.0,1270.0
...,...,...,...,...,...,...,...,...
90875,50,Entity_98,Class_3,Prop_5,579.0,I,758.7,526.1
90876,50,Entity_98,Class_3,Prop_5,579.0,R,667.7,891.0
90877,50,Entity_98,Class_3,Prop_5,579.0,S,459.4,659.7
90878,50,Entity_98,Class_3,Prop_5,579.0,V,1039.0,464.6


## 🔍 Value kinds must be treated separately!

The dataset contains time series data for **multiple value kinds** (e.g., "A", "B", "C", ...).  
These represent **distinct types of values**, each with their own internal dynamics.

Each `value_kind` is a different thing and it is incorrect to aggregate values of different kinds. Also, in all likelighood different `value_kind`s behave differently over time as well.

We will **split the dataset into separate subsets**, one for each `value_kind`.  
From here on, **all modeling, analysis, and evaluation must happen separately per value kind.**

This ensures our models capture the true, unblended behavior of each type of value.

In [ ]:
# Check how many value kinds exist
unique_vks = df['value_kind'].unique()
print(f"Number of value kinds: {len(unique_vks)}")

# Split into one DataFrame per value kind
value_kind_datasets = {
    vk: df[df['value_kind'] == vk].copy().reset_index(drop=True)
    for vk in unique_vks
}

# Example: show the data from value kind "A"
value_kind_datasets['A']

Number of value kinds: 24


,time_index,entity_name,entity_class,property_class,size,value_kind,target_1,target_2
0,1,Entity_22,Class_1,Prop_1,660.0,A,1157.0,1495.0
1,1,Entity_299,Class_7,Prop_5,673.0,A,642.5,669.6
2,2,Entity_22,Class_1,Prop_1,702.0,A,1343.0,1576.0
3,2,Entity_29,Class_1,Prop_5,2116.0,A,4777.0,4755.0
4,2,Entity_299,Class_7,Prop_5,805.0,A,716.4,624.8
...,...,...,...,...,...,...,...,...
3941,50,Entity_94,Class_3,Prop_2,1653.0,A,2779.0,1252.0
3942,50,Entity_95,Class_3,Prop_5,3114.0,A,4516.0,2432.0
3943,50,Entity_96,Class_3,Prop_2,75676.0,A,110810.0,54448.0
3944,50,Entity_97,Class_3,Prop_1,757.0,A,1193.0,520.2


This will give you a dictionary `value_kind_datasets` where:
- `value_kind_datasets['A']` contains only rows for value kind "A".
- ... and so on for each value kind

## Storage efficiency

So far, our data has been stored as a single CSV file. While this works for small datasets, it becomes **inefficient and slow** as the dataset grows larger or more complex.

### Limitations of CSV files:
- CSV is a plain-text format, so file sizes are **large**.
- It doesn't preserve **data types** (e.g., integers vs floats).
- Loading a large CSV can be **slow**, especially in Jupyter notebooks.
- It lacks support for **column indexing**, **compression**, or **partial loading**.

### Enter Parquet:
Parquet is a **binary columnar format** optimized for efficient analytics.

- **Faster to read/write**
- **Much smaller file size** due to compression
- **Preserves data types**
- Ideal for structured tabular data and scalable ML pipelines

### One file per value kind

- Each `value_kind` represents a **distinct kind of thing**, and we usually want to **analyze or model them separately**.
- Saving them to **separate files** allows us to:
  - Load only the value kinds we need,
  - Reduce memory usage,
  - Parallelize processing across different value kinds.

This structure aligns with our modeling strategy — one model per value kind — and allows for more efficient exploration.

In [ ]:
# Create a folder to hold the files
output_dir = os.path.join(base_path, "value_kind_datasets")
os.makedirs(output_dir, exist_ok=True)

# Save each value kind to a separate .parquet file
for vk, df_vk in value_kind_datasets.items():
    filename = os.path.join(output_dir, f"value_kind_{vk}.parquet")
    df_vk.to_parquet(filename, index=False)

print(f"Saved {len(value_kind_datasets)} value kinds to: {output_dir}/value_kind_*.parquet")

Saved 24 value kinds to: /content/drive/MyDrive/Colab Notebooks/CMPT 3510 W25/Project/value_kind_datasets/value_kind_*.parquet


## Exploring the data

Now that we’ve split the dataset by `value_kind`, we can examine how the **ratio** of the two targets evolves over time for different entity classes.

This ratio is defined as:

$$
\text{ratio} = \frac{\text{target}_1}{\text{target}_2}
$$

This is the **key metric** we want to evaluate:  
Our goal is to find the **entity class** with the **lowest overall ratio**.

Entity classes produce different kinds of values and behave differently **within each value kind**. This plot will help us visually inspect how different entity classes compare over time for a **single value kind**.

Use the variable below to pick a value kind of interest and observe how classes differ!

In [ ]:
# 👇 Change this to explore different value kinds
selected_value_kind = 'A'

In [ ]:
# Extract corresponding dataframe
df_vk = value_kind_datasets[selected_value_kind].copy()

# Compute ratio if not already present
if 'ratio' not in df_vk.columns:
    df_vk['ratio'] = df_vk['target_1'] / df_vk['target_2']

# Compute average ratio per class and time
avg_ratio_by_class = (
    df_vk.groupby(['entity_class', 'time_index'])['ratio']
    .mean()
    .reset_index()
)

# Interactive line plot
fig = px.line(
    avg_ratio_by_class,
    x='time_index',
    y='ratio',
    color='entity_class',
    title=f'Average ratio over time by entity class — value kind {selected_value_kind}',
    labels={'ratio': 'target₁ / target₂', 'time_index': 'Time index'},
    width=1100, height=600
)
fig.show()

### Final average ratio by entity class

Let’s compute and compare the **overall average ratio** per entity class for the selected `value_kind`.

This helps us see **which classes tend to have higher or lower ratios** across all their entities and time points.

Remember, our end goal is to **identify the class with the lowest ratio** — this plot gives us a first hint.

In [ ]:
# Compute final mean ratio per class
class_avg_ratio = df_vk.groupby('entity_class')['ratio'].mean().reset_index()

fig = px.bar(
    class_avg_ratio.sort_values(by='ratio'),
    x='entity_class',
    y='ratio',
    title=f'Average ratio per entity class — value kind {selected_value_kind}',
    labels={'ratio': 'Mean(target₁ / target₂)'},
    height=500
)
fig.show()

### Entity-level ratio trends within a single class

Here we explore **variation across individual entities** in a selected class.

Even within the same class, entities can behave quite differently — this helps us think about **variance**, outliers, and whether we should model individual entities or use class-level summaries.

In [ ]:
# Choose a class (students can modify this)
selected_class = df_vk['entity_class'].unique()[0]

subset = df_vk[df_vk['entity_class'] == selected_class]

fig = px.line(
    subset,
    x='time_index',
    y='ratio',
    color='entity_name',
    title=f'Ratio over time for entities in class {selected_class} — value kind {selected_value_kind}',
    labels={'ratio': 'target₁ / target₂'},
    height=600
)
fig.show()

### Comparing size trends across two classes

Do different entity classes grow differently in size?

This may help us understand whether **size dynamics** play a role in determining the ratio.

Let’s compare size trends across two selected classes.

In [ ]:
# Choose two classes
class_1, class_2 = df_vk['entity_class'].unique()[:2]

size_trends = (
    df_vk[df_vk['entity_class'].isin([class_1, class_2])]
    .groupby(['entity_class', 'time_index'])['size']
    .mean()
    .reset_index()
)

fig = px.line(
    size_trends,
    x='time_index',
    y='size',
    color='entity_class',
    title=f'Mean entity size over time — classes {class_1} vs {class_2}',
    labels={'size': 'Mean Size'},
    height=500
)
fig.show()

### Correlation between size and target values

It’s important to understand how **entity size** affects the targets.

Here we plot both target values against size, using scatter plots to show whether there’s a linear or nonlinear relationship.

In [ ]:
fig = px.scatter(
    df_vk,
    x='size',
    y='target_1',
    color='entity_class',
    title=f'Size vs target 1 — value kind {selected_value_kind}',
    opacity=0.6,
    height=500
)
fig.show()

fig = px.scatter(
    df_vk,
    x='size',
    y='target_2',
    color='entity_class',
    title=f'Size vs target 2 — value kind {selected_value_kind}',
    opacity=0.6,
    height=500
)
fig.show()

### Distribution of target values

What are the scales and spreads of the target values?

This helps us understand whether we need normalization, log transforms, or robust statistics.

In [ ]:
fig = px.histogram(
    df_vk,
    x='target_1',
    color='entity_class',
    marginal='box',
    title=f'Distribution of target_1 — value kind {selected_value_kind}',
    nbins=50,
    height=500
)
fig.show()

fig = px.histogram(
    df_vk,
    x='target_2',
    color='entity_class',
    marginal='box',
    title=f'Distribution of target_2 —  value kind {selected_value_kind}',
    nbins=50,
    height=500
)
fig.show()

## Time series prediction: a sequence modeling problem

We’ve explored the dataset, seen that it evolves over time, and visualized how our targets (`target_1` and `target_2`) change.

Now we shift our focus to **prediction** — but this is not just any kind of prediction task. It’s a special case of **sequence prediction**, specifically **time series forecasting**.

### What is sequence prediction?

In **sequence prediction**, we try to predict the **next element** in a sequence based on its **previous elements**.

- Input: a sequence of past observations
- Output: the next item (or items) in that sequence

Familiar examples include:
- Predicting the next word in a sentence (language modeling)
- Predicting the next frame in a video
- Predicting the next number in a sensor reading or stock price

Time series prediction is a type of **numerical sequence prediction**, where the values evolve over time and may be influenced by other features.

### Our goal

We want to predict future values of a target variable like `target_1`, using past information.  
This puts us squarely in the domain of **time series prediction**, a structured form of supervised learning that takes sequence information into account.

### What makes time series prediction different?

In classical machine learning, we often think of a dataset as a table where each row is **independent** of others:

| feature₁ | feature₂ | ... | target |
|----------|----------|-----|--------|
| ...      | ...      | ... | ...    |

In time series prediction, this assumption no longer holds. Each observation **depends heavily on the past**.

The most important feature in forecasting is usually the **historical values of the target itself**. These are called **lags**.

>**Most predictive information comes from: $y_{t-1},\ y_{t-2},\ \dots,\ y_{t-\tau}$**

This is what sets time series prediction apart from typical ML:
- In typical ML, we only have **exogenous variables** (external features),
- In time series ML, we have **lagged targets**, which are the **most informative** features.

> **Important**: The most valuable predictors in time series are usually the **past values of the target you're trying to forecast.**  
> Exogenous variables — like size, class, or category — are still useful, but they play a **secondary** role.

### General formulation

In time series prediction, we model the future value$y_t$ based on historical patterns:

$$
y_t = f(y_{t-1}, y_{t-2}, \dots, y_{t-\tau},\ x_{t-1}, x_{t-2}, \dots, x_{t-\tau})
$$

Where:

- $y_t$: the **target** at time$t$,
- $\tau$: the **lookback window** or forecasting horizon,
- $x_t$: the **exogenous features** — features that are not target values (e.g., size, entity class),
- $f$: the **forecasting function**, which can be:
  - Classical (e.g., ARIMA, exponential smoothing)
  - Machine learning-based (e.g., Random Forest, XGBoost)
  - Neural network-based (e.g., LSTM, Transformer)

The core idea is:  
> You want to **extract enough past context** to make an informed prediction about what happens next.

In the next section, we’ll explore how to **restructure our dataset** so that each row corresponds to a supervised ML example with:
- Lagged values of the target
- Lagged or current values of exogenous variables
- A target value at time $t$

This will allow us to apply **standard ML techniques** to a time series setting.

### Terminology

| Term                | Meaning                                                                 |
|---------------------|-------------------------------------------------------------------------|
| **Lag**             | A past value of a variable, e.g., $y_{t-1}$                         |
| **Exogenous variable** | A feature **not derived** from the target time series                |
| **Autoregressive**  | A model that predicts based on **past values of the target itself**     |
| **Forecast horizon**| How far ahead (or back) we look, usually $\tau$                     |
| **Univariate**      | Modeling a single time series                                           |
| **Multivariate**    | Modeling multiple related time series                                   |
| **One-step vs Multi-step** | Predicting only the next time step vs multiple future steps      |

---
**Note**: Reading this next subsection (*Classical time series modeling approaches*) is optional but recommended.


### Classical time series modeling approaches

There are several standard approaches to time series forecasting. These methods are based on statistical assumptions and often provide strong baselines — especially for univariate time series with regular patterns.

#### AR / MA / ARMA / ARIMA

- **AR (Autoregressive)**: Uses past values of the series $y_{t-1}, y_{t-2}, \dots$ to predict $y_t$.  
 $$
  y_t = \phi_1 y_{t-1} + \phi_2 y_{t-2} + \dots + \epsilon_t
 $$

- **MA (Moving Average)**: Models $y_t$ as a function of past **forecast errors**:
 $$
  y_t = \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots
 $$

- **ARMA**: Combines both AR and MA. Assumes the time series is **stationary**.

- **ARIMA (AutoRegressive Integrated Moving Average)**: Extends ARMA by adding **differencing** to handle **non-stationary** series:
 $$
  \text{Apply } \Delta y_t = y_t - y_{t-1}, \quad \text{then model with ARMA}
 $$

#### ARIMAX (ARIMA with eXogenous variables)

- Like ARIMA, but also includes **external (exogenous) variables** $x_t$ that might help predict the target:
 $$
  y_t = \phi_1 y_{t-1} + \dots + \beta_1 x_{t-1} + \dots + \epsilon_t
 $$
- Useful when external factors (like economic indicators, weather, or entity size in our case) affect the series.

#### ARMAX (ARMA with eXogenous variables)

- Similar to ARIMAX, but assumes the original series is stationary — no differencing required.
- Includes autoregression + moving average + exogenous regressors.

#### SARIMA (Seasonal ARIMA)

- Extends ARIMA by modeling **seasonality**:
 $$
  SARIMA(p, d, q)(P, D, Q)_s
 $$
  where the second group of parameters handles seasonal components with period $s$ (e.g., 12 for months, 24 for hours).
- Captures repeating cycles, like daily or yearly patterns.

#### SARIMAX (Seasonal ARIMAX)

- Combines ARIMA, **seasonality**, and **exogenous variables** — the most general and powerful form in this family.
- Can be written as:
 $$
  y_t = \text{seasonal terms} + \text{ARIMA terms} + \text{exogenous terms} + \epsilon_t
 $$
- Often used in business and econometrics when both internal cycles and external influences matter.

#### Exponential Smoothing (ETS)

- Gives **more weight to recent observations**, using a recursive formula:
 $$
  \hat{y}_{t+1} = \alpha y_t + (1 - \alpha) \hat{y}_t
 $$
- Extensions like **Holt-Winters** handle trend and seasonality.
- Works well when noise is low and structure is strong.

#### State-space models (e.g., Kalman filters)

- Model a **latent state** that evolves over time and emits observable outputs.
- Capture uncertainty, measurement noise, and system dynamics.
- Can incorporate **exogenous variables** in a principled way.

#### VAR (Vector AutoRegression)

- A multivariate version of AR: models each variable using **lagged values of itself and other variables**.
- Good for systems with many interacting series (e.g., economic indicators, sensor networks).

#### Gaussian processes for time series

- A fully Bayesian, non-parametric approach.
- Models correlations between observations using kernels.
- Powerful but computationally expensive for large data.

---

###  Can we use plain machine learning?

Yes — but we must **transform the dataset** into a form that ML models like!

Most ML models (e.g., decision trees, gradient boosting, linear regression) expect **tabular input**:

$$
\text{Features} \Rightarrow \text{Target}
$$

So we must **synthesize this structure** ourselves.

### Synthesizing a forecasting dataset for ML

To do this, we construct rows where each one contains:

1. **Lagged values of the target**:
   $$
   \text{Feature columns: } y_{t-1}, y_{t-2}, \dots, y_{t-\tau}
   $$

2. **Current or past values of exogenous variables**:
   - Size at time $t-1$
   - Entity class
   - Property class
   - Value kind (if modeling jointly)
   - Possibly lagged exogenous values too, like size$_{t-2}$, etc.

3. **Target value at time $t$**:
   $$
   y_t \quad \text{(what we want to predict)}
   $$

#### Example row structure for ML

| yₜ₋₁ | yₜ₋₂ | yₜ₋₃ | sizeₜ₋₁ | entity_class | property_class | targetₜ |
|------|------|------|----------|---------------|------------------|----------|
| 10.2 | 9.7  | 10.5 | 122      | Class_1        | Prop_2           | 11.0     |

Each row is **an independent example** for the ML model. This process is called **sliding window generation** and gives us a classic **supervised learning** dataset.


### Important constraints

- You need **at least $\tau$ past time steps** to create one row — so the early parts of time series will be skipped.
- All features must be **aligned correctly in time** — your model should never "see the future".

### Your task as a feature engineer

Think carefully:

- What are **useful features** for prediction?
- What values are **available at training time**?
- Should you use **only target lags**, or also **lagged exogenous features**?

We will now write code to construct such a dataset.

## Constructing features for time series ML

To use machine learning models for forecasting, we need to **reshape our time series into a supervised learning format**.

Each **row** in the new dataset will represent:
- The **past $\tau$ values** of `target_1` for an entity
- The **exogenous variables**:
  - Entity size at the current time
  - Entity class (categorical)
  - Property class (categorical)

And the **label (target)** will be the value of `target_1` at the current time.

### What we’re building

We choose a fixed lag $\tau = 5$.  
For every valid time index $t$, we will build rows of the form:

$$
\left( y_{t-5},\ y_{t-4},\ y_{t-3},\ y_{t-2},\ y_{t-1},\ \text{size}_t,\ \text{entity\_class},\ \text{property\_class} \right) \Rightarrow y_t
$$

This is a classic **sliding window** approach.

Each such row becomes one supervised training example.

> Note: The first few time steps (e.g., $t = 1$ to $t = 5$) **cannot be used** since we don’t have enough past values yet.

In [ ]:
# 👇 Select value kind to work on
selected_value_kind = 'A'

In [ ]:
# Set horizon
tau = 5  # lag horizon

In [ ]:
df_vk = value_kind_datasets[selected_value_kind].copy()

# Sort to ensure time-ordering within each entity
df_vk = df_vk.sort_values(by=['entity_name', 'time_index'])

# Store feature rows
feature_rows = []

# Build rows entity-by-entity
for entity, group in df_vk.groupby('entity_name'):
    group = group.reset_index(drop=True)

    # Skip short sequences
    if len(group) <= tau:
        continue

    for i in range(tau, len(group)):
        row = group.iloc[i]

        # Extract lag features for both targets
        t1_lags = group.iloc[i - tau:i]['target_1'].tolist()
        t2_lags = group.iloc[i - tau:i]['target_2'].tolist()

        # Use current time values
        size = row['size']
        entity_class = row['entity_class']
        property_class = row['property_class']
        time = row['time_index']
        t1 = row['target_1']
        t2 = row['target_2']

        feature_rows.append({
            'time_index': time,
            'entity_name': entity,
            'entity_class': entity_class,
            'property_class': property_class,
            'size': size,
            **{f"target_1_lag_{j+1}": t1_lags[j] for j in range(tau)},
            **{f"target_2_lag_{j+1}": t2_lags[j] for j in range(tau)},
            'target_1': t1,
            'target_2': t2
        })

# Create final DataFrame
ml_df = pd.DataFrame(feature_rows)
ml_df

,time_index,entity_name,entity_class,property_class,size,target_1_lag_1,target_1_lag_2,target_1_lag_3,target_1_lag_4,target_1_lag_5,target_2_lag_1,target_2_lag_2,target_2_lag_3,target_2_lag_4,target_2_lag_5,target_1,target_2
0,21,Entity_1,Class_1,Prop_3,1620.0,2625.0,2564.0,2352.0,2411.0,2208.0,2588.0,2918.0,2856.0,2695.0,3196.0,2667.0,2983.0
1,22,Entity_1,Class_1,Prop_3,1958.0,2564.0,2352.0,2411.0,2208.0,2667.0,2918.0,2856.0,2695.0,3196.0,2983.0,3540.0,4031.0
2,23,Entity_1,Class_1,Prop_3,2117.0,2352.0,2411.0,2208.0,2667.0,3540.0,2856.0,2695.0,3196.0,2983.0,4031.0,3147.0,3997.0
3,24,Entity_1,Class_1,Prop_3,2584.0,2411.0,2208.0,2667.0,3540.0,3147.0,2695.0,3196.0,2983.0,4031.0,3997.0,3968.0,7182.0
4,25,Entity_1,Class_1,Prop_3,2582.0,2208.0,2667.0,3540.0,3147.0,3968.0,3196.0,2983.0,4031.0,3997.0,7182.0,4649.0,5473.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,50,Entity_98,Class_3,Prop_5,579.0,752.9,818.3,898.3,1033.0,1044.0,380.9,438.3,515.9,553.6,537.4,924.4,416.8
3186,34,Entity_99,Class_3,Prop_3,1333.0,1013.0,1417.0,1590.0,1687.0,1441.0,600.0,673.6,562.8,634.8,872.1,2225.0,1187.0
3187,35,Entity_99,Class_3,Prop_3,1383.0,1417.0,1590.0,1687.0,1441.0,2225.0,673.6,562.8,634.8,872.1,1187.0,2209.0,1230.0
3188,36,Entity_99,Class_3,Prop_3,1581.0,1590.0,1687.0,1441.0,2225.0,2209.0,562.8,634.8,872.1,1187.0,1230.0,2634.0,1134.0


### Wait! We made a mistake (and that’s a good thing)

In our last step, we created a dataset that uses:
- The past 5 values of `target_1` ✅ (correct)
- The **current time step's value** of the `size` column ❌ (not correct!)
- The `entity_class` and `property_class` ✅ (conditionally okay)

#### Why is this a problem?

This is a classic mistake in time series prediction:

> **We used a value (`size_t`) from the future of our feature window.**

That means: we used the entity's size at time $t$ to help predict the target at time $t$.  
But we won’t have access to that value when we’re actually making a prediction!

#### Time series is causal

In time series forecasting, you can **only use information available up to time $t-1$** when predicting $y_t$.

Your features must come from:
- Past targets: $y_{t-1}, y_{t-2}, \dots$
- Past exogenous variables: $x_{t-1}, x_{t-2}, \dots$

Otherwise, you’re peeking into the future – and that breaks the core principle of forecasting.

### Lesson: think about what you actually know at prediction time

When constructing features, ask:

1. **Will I know this feature’s value at prediction time?**
2. **If someone gave me these features, would I be able to make a useful guess myself?**
3. **Would I be using future information, even accidentally?**

If your answer to (1) is "no" — the model shouldn’t be allowed to use it either.

### What can we use?

Let’s analyze:

- **Past values of the target**: ✅ Always fair game.
- **Past values of size**: ✅ Yes, we can use `size_{t-1}`, `size_{t-2}`, etc.
- **Size at time $t$**: ❌ No, we don’t know it yet.
- **Entity class**: ✅ It doesn't change over time. Known at all times.
- **Property class**: ✅ Also static. Known at all times.

### Bonus insight: use size history to estimate current size

Even though we don’t know the exact value of size at time $t$, we can **infer it** using its past values.

There are two approaches:

1. **Explicit modeling**: Fit a size prediction model separately, then use the predicted size.
2. **Implicit modeling**: Just feed the past size values (e.g., `size_{t-1}, ..., size_{t-5}`) into the model and let it learn to estimate the pattern on its own.

We’ll take the second approach for now — it requires less engineering and lets the ML model learn growth dynamics from the data directly.

> So our final input features will include:
> - Lagged target values: $y_{t-1}, \dots, y_{t-\tau}$
> - Lagged size values: $\text{size}_{t-1}, \dots, \text{size}_{t-\tau}$
> - Static features: entity class, property class

Let’s now rebuild the dataset the **correct** way.

### Real-world time series are not perfect

In our previous code, we built lagged features by walking through the rows of a DataFrame and manually extracting the $\tau$ most recent rows using `.iloc`.

This works only if:

- Each entity has **consecutive time indices** with **no gaps**,
- The DataFrame is sorted perfectly and behaves like an array.

But in real-world settings — and even in our synthetic dataset — this assumption may not always hold.

#### What can go wrong?

Imagine an entity has data for:

$$
t = [1, 2, 4, 5, 6]
$$

Then:
- At $t = 6$, if we ask for the last 5 time steps using `.iloc[i-5:i]`, we get:
  - $t = 1, 2, 4, 5, 6$ → wrong ordering, and also $t = 3$ is missing
  - **These are not the actual previous consecutive time steps**.

Even worse, we might end up grabbing **values from the wrong times entirely**.

#### Why `shift()` doesn’t help

Pandas has a handy `.shift()` function to build lag features:

```python
df['y_lag_1'] = df['y'].shift(1)
```

But this assumes:
- The DataFrame is already **grouped and sorted perfectly** by entity and time,
- There are **no missing time steps**, or else `.shift()` gives you lagged values from the wrong point in time.

### The real fix: index by entity and time

The robust way to construct lagged features is to:

1. Create a **multi-index** on `entity_name` and `time_index`,
2. Use `.loc` to look up exact previous time indices (e.g., $t-1, t-2, \dots$),
3. Check for missing lags and **drop those rows**.

This way:
- You’re always getting the **correct historical value**, not just the previous row.
- Your code works **even when time steps are missing or misaligned**.

> This is how we’ll do it in the corrected code.

We’ll now rebuild the ML-ready dataset using **lagged lookups** based on `entity_name` and `time_index`, ensuring consistency even when time steps are irregular.

In [ ]:
# Extract and prepare data
df_vk = value_kind_datasets[selected_value_kind].copy()
df_vk = df_vk.sort_values(by=['entity_name', 'time_index'])
df_vk.set_index(['entity_name', 'time_index'], inplace=True)

# Container for output
feature_rows = []

# Loop through every row
for entity, time in df_vk.index:
    has_all_lags = True
    row_features = {}

    # Collect lags for both targets and size
    target_1_lags = []
    target_2_lags = []
    size_lags = []

    for lag in range(1, tau + 1):
        key = (entity, time - lag)
        if key in df_vk.index:
            past_row = df_vk.loc[key]
            target_1_lags.append(past_row['target_1'])
            target_2_lags.append(past_row['target_2'])
            size_lags.append(past_row['size'])
        else:
            has_all_lags = False
            break

    if not has_all_lags:
        continue

    # Get current row values
    current_row = df_vk.loc[(entity, time)]
    t1 = current_row['target_1']
    t2 = current_row['target_2']
    entity_class = current_row['entity_class']
    property_class = current_row['property_class']

    # Build the feature dictionary
    row = {
        'time_index': time,
        'entity_name': entity,
        'entity_class': entity_class,
        'property_class': property_class,
        **{f'size_lag_{i+1}': size_lags[i] for i in range(tau)},
        **{f'target_1_lag_{i+1}': target_1_lags[i] for i in range(tau)},
        **{f'target_2_lag_{i+1}': target_2_lags[i] for i in range(tau)},
        'target_1': t1,
        'target_2': t2,
    }

    feature_rows.append(row)

# Final ML-friendly DataFrame
ml_df = pd.DataFrame(feature_rows)
ml_df

,time_index,entity_name,entity_class,property_class,size_lag_1,size_lag_2,size_lag_3,size_lag_4,size_lag_5,target_1_lag_1,...,target_1_lag_3,target_1_lag_4,target_1_lag_5,target_2_lag_1,target_2_lag_2,target_2_lag_3,target_2_lag_4,target_2_lag_5,target_1,target_2
0,21,Entity_1,Class_1,Prop_3,1385.0,1379.0,1328.0,1230.0,1343.0,2208.0,...,2352.0,2564.0,2625.0,3196.0,2695.0,2856.0,2918.0,2588.0,2667.0,2983.0
1,22,Entity_1,Class_1,Prop_3,1620.0,1385.0,1379.0,1328.0,1230.0,2667.0,...,2411.0,2352.0,2564.0,2983.0,3196.0,2695.0,2856.0,2918.0,3540.0,4031.0
2,23,Entity_1,Class_1,Prop_3,1958.0,1620.0,1385.0,1379.0,1328.0,3540.0,...,2208.0,2411.0,2352.0,4031.0,2983.0,3196.0,2695.0,2856.0,3147.0,3997.0
3,24,Entity_1,Class_1,Prop_3,2117.0,1958.0,1620.0,1385.0,1379.0,3147.0,...,2667.0,2208.0,2411.0,3997.0,4031.0,2983.0,3196.0,2695.0,3968.0,7182.0
4,25,Entity_1,Class_1,Prop_3,2584.0,2117.0,1958.0,1620.0,1385.0,3968.0,...,3540.0,2667.0,2208.0,7182.0,3997.0,4031.0,2983.0,3196.0,4649.0,5473.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,50,Entity_98,Class_3,Prop_5,731.0,694.0,600.0,520.0,501.0,1044.0,...,898.3,818.3,752.9,537.4,553.6,515.9,438.3,380.9,924.4,416.8
3186,34,Entity_99,Class_3,Prop_3,1006.0,980.0,917.0,796.0,727.0,1441.0,...,1590.0,1417.0,1013.0,872.1,634.8,562.8,673.6,600.0,2225.0,1187.0
3187,35,Entity_99,Class_3,Prop_3,1333.0,1006.0,980.0,917.0,796.0,2225.0,...,1687.0,1590.0,1417.0,1187.0,872.1,634.8,562.8,673.6,2209.0,1230.0
3188,36,Entity_99,Class_3,Prop_3,1383.0,1333.0,1006.0,980.0,917.0,2209.0,...,1441.0,1687.0,1590.0,1230.0,1187.0,872.1,634.8,562.8,2634.0,1134.0


## Predicting the ratio: two options

Our end goal is to understand the behavior of the **ratio** between two targets:

$$
\text{ratio} = \frac{\text{target}_1}{\text{target}_2}
$$

There are two ways to approach this:

### Option 1: predict the ratio directly

We can train a model to predict the ratio as its own target variable.

**Pros:**
- Directly optimizes for what we care about.
- Avoids the compounded error of dividing two model outputs.
- Useful when we're only interested in ranking or classification based on the ratio.

**Cons:**
- Ignores the structure and meaning of the individual targets.
- Can be unstable if the ratio has high variance or noise.
- Loses interpretability: we don't know what contributes to high or low ratios.

### Option 2: Predict `target_1` and `target_2` separately, then compute the ratio

We can train two separate models to predict each target individually, then compute the ratio from their predictions.

**Pros:**
- Preserves more information.
- Each target can be modeled according to its own dynamics.
- More interpretable — we can see which target is driving the ratio.
- More flexible if we ever need the raw predictions.

**Cons:**
- The resulting ratio prediction is not explicitly optimized.
- Errors can compound nonlinearly when taking the division.

### Our Choice

We will go with **option 2** — predicting `target_1` and `target_2` separately and then computing the ratio from their predictions.

This gives us more flexibility and interpretability, and lets us work with richer information for downstream analysis.

## Two supervised learning problems

Since we’ve chosen to **predict each target separately**, we will now split our dataset into two parts:

- One for predicting `target_1`
- One for predicting `target_2`

Each dataset will contain:

- The lagged values of the corresponding target,
- Lagged values of `size`,
- Entity metadata (`entity_class`, `property_class`) as static exogenous features,
- The associated label: either `target_1` or `target_2`

At this point, each dataset becomes a **classic regression problem**:

- You have a matrix of features,
- And a continuous label to predict.

This means we can apply all the usual **ML data exploration techniques**, including:
- Plotting histograms of the target,
- Checking correlation with lagged features,
- Grouping by categorical variables to explore variation,
- Identifying skewness or outliers.

We’ll explore these patterns before building prediction models.

In [ ]:
# Create dataset for predicting target_1
ml_df_target_1 = ml_df[[
    'time_index', 'entity_name', 'entity_class', 'property_class',
    'target_1'
] + [col for col in ml_df.columns if col.startswith('target_1_lag_') or col.startswith('size_lag_')]].copy()

# Rename label column and move to end
ml_df_target_1 = ml_df_target_1.rename(columns={'target_1': 'target'})
cols_1 = [col for col in ml_df_target_1.columns if col != 'target'] + ['target']
ml_df_target_1 = ml_df_target_1[cols_1]

# Create dataset for predicting target_2
ml_df_target_2 = ml_df[[
    'time_index', 'entity_name', 'entity_class', 'property_class',
    'target_2'
] + [col for col in ml_df.columns if col.startswith('target_2_lag_') or col.startswith('size_lag_')]].copy()

# Rename label column and move to end
ml_df_target_2 = ml_df_target_2.rename(columns={'target_2': 'target'})
cols_2 = [col for col in ml_df_target_2.columns if col != 'target'] + ['target']
ml_df_target_2 = ml_df_target_2[cols_2]

# Check result
ml_df_target_1

,time_index,entity_name,entity_class,property_class,size_lag_1,size_lag_2,size_lag_3,size_lag_4,size_lag_5,target_1_lag_1,target_1_lag_2,target_1_lag_3,target_1_lag_4,target_1_lag_5,target
0,21,Entity_1,Class_1,Prop_3,1385.0,1379.0,1328.0,1230.0,1343.0,2208.0,2411.0,2352.0,2564.0,2625.0,2667.0
1,22,Entity_1,Class_1,Prop_3,1620.0,1385.0,1379.0,1328.0,1230.0,2667.0,2208.0,2411.0,2352.0,2564.0,3540.0
2,23,Entity_1,Class_1,Prop_3,1958.0,1620.0,1385.0,1379.0,1328.0,3540.0,2667.0,2208.0,2411.0,2352.0,3147.0
3,24,Entity_1,Class_1,Prop_3,2117.0,1958.0,1620.0,1385.0,1379.0,3147.0,3540.0,2667.0,2208.0,2411.0,3968.0
4,25,Entity_1,Class_1,Prop_3,2584.0,2117.0,1958.0,1620.0,1385.0,3968.0,3147.0,3540.0,2667.0,2208.0,4649.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,50,Entity_98,Class_3,Prop_5,731.0,694.0,600.0,520.0,501.0,1044.0,1033.0,898.3,818.3,752.9,924.4
3186,34,Entity_99,Class_3,Prop_3,1006.0,980.0,917.0,796.0,727.0,1441.0,1687.0,1590.0,1417.0,1013.0,2225.0
3187,35,Entity_99,Class_3,Prop_3,1333.0,1006.0,980.0,917.0,796.0,2225.0,1441.0,1687.0,1590.0,1417.0,2209.0
3188,36,Entity_99,Class_3,Prop_3,1383.0,1333.0,1006.0,980.0,917.0,2209.0,2225.0,1441.0,1687.0,1590.0,2634.0


## Exploratory Data Analysis (EDA)

Now that `ml_df_target_1` is structured like a classic regression dataset, we can begin by exploring its properties.

We’ll look at:

- The distribution of the target,
- The relationships between lagged features and the target,
- Differences across entity classes and property classes,
- Basic statistics for the lagged features.

This step is essential to build **intuition** about the patterns and potential difficulties in prediction.

In [ ]:
fig = px.histogram(ml_df_target_1, x="target", nbins=50, title="Distribution of target 1")
fig.show()

In [ ]:
px.scatter(
    ml_df_target_1,
    x="target_1_lag_1",
    y="target",
    trendline="ols",
    title="Target vs most recent lag (lag 1)"
).show()

In [ ]:
px.box(
    ml_df_target_1,
    x="entity_class",
    y="target",
    title="Target 1 distribution by entity class"
).show()

In [ ]:
px.box(
    ml_df_target_1,
    x="property_class",
    y="target",
    title="Target 1 distribution by property class"
).show()

To understand the internal structure of our feature space, we now look at:

1. **Histogram of all features**
- Helps reveal distributions, skewness, or anomalies.
- We will have box plots on the margin of histograms for numerical features: while histograms show the shape of the distribution, **box plots** give a quick overview of:

 - Minimum and maximum values
 - Median and interquartile range (IQR)
 - Outliers (as individual points)
 - Variance and skewness at a glance

2. **Scatterplot matrix**
- Visual overview of pairwise linear or non-linear patterns between features and target.

3. **Correlation heatmap**
- Identifies multicollinearity (strong correlation between input features),
- Highlights the most influential features with respect to the target.

In [ ]:
# Display each numeric feature as histogram + marginal box
numeric_cols = [col for col in ml_df_target_1.columns if col.startswith(('target_', 'size_lag_')) or col == 'target']

for col in numeric_cols:
    fig = px.histogram(
        ml_df_target_1,
        x=col,
        nbins=40,
        marginal='box',
        title=f"Distribution of {col} with box plot on margin"
    )
    fig.update_layout(width=900, height=400)
    fig.show()

In [ ]:
fig = px.scatter_matrix(
    ml_df_target_1,
    dimensions=numeric_cols,
    color="entity_class",
    title="Scatterplot matrix of features",
)

# Remove diagonal histograms and adjust size
fig.update_traces(diagonal_visible=False)
fig.update_layout(
    width=1200,
    height=1200,
    dragmode='select',
    margin=dict(l=50, r=50, b=50, t=80),
)

fig.show()

In [ ]:
# Calculate correlation matrix
corr_matrix = ml_df_target_1[numeric_cols].corr()

# Create heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdBu',
        zmin=-1,
        zmax=1,
        colorbar=dict(title='Correlation')
    )
)

fig.update_layout(
    title='Correlation heatmap of numeric features',
    xaxis_nticks=len(corr_matrix.columns),
    yaxis_nticks=len(corr_matrix.columns),
    xaxis_title='Features',
    yaxis_title='Features',
    autosize=False,
    width=900,
    height=800
)

fig.show()

Lets' review our dataset once more:

In [ ]:
ml_df_target_1

,time_index,entity_name,entity_class,property_class,size_lag_1,size_lag_2,size_lag_3,size_lag_4,size_lag_5,target_1_lag_1,target_1_lag_2,target_1_lag_3,target_1_lag_4,target_1_lag_5,target
0,21,Entity_1,Class_1,Prop_3,1385.0,1379.0,1328.0,1230.0,1343.0,2208.0,2411.0,2352.0,2564.0,2625.0,2667.0
1,22,Entity_1,Class_1,Prop_3,1620.0,1385.0,1379.0,1328.0,1230.0,2667.0,2208.0,2411.0,2352.0,2564.0,3540.0
2,23,Entity_1,Class_1,Prop_3,1958.0,1620.0,1385.0,1379.0,1328.0,3540.0,2667.0,2208.0,2411.0,2352.0,3147.0
3,24,Entity_1,Class_1,Prop_3,2117.0,1958.0,1620.0,1385.0,1379.0,3147.0,3540.0,2667.0,2208.0,2411.0,3968.0
4,25,Entity_1,Class_1,Prop_3,2584.0,2117.0,1958.0,1620.0,1385.0,3968.0,3147.0,3540.0,2667.0,2208.0,4649.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,50,Entity_98,Class_3,Prop_5,731.0,694.0,600.0,520.0,501.0,1044.0,1033.0,898.3,818.3,752.9,924.4
3186,34,Entity_99,Class_3,Prop_3,1006.0,980.0,917.0,796.0,727.0,1441.0,1687.0,1590.0,1417.0,1013.0,2225.0
3187,35,Entity_99,Class_3,Prop_3,1333.0,1006.0,980.0,917.0,796.0,2225.0,1441.0,1687.0,1590.0,1417.0,2209.0
3188,36,Entity_99,Class_3,Prop_3,1383.0,1333.0,1006.0,980.0,917.0,2209.0,2225.0,1441.0,1687.0,1590.0,2634.0


## Feature selection and categorical encoding

Before training, we need to prepare our dataset by:

1. **Removing non-informative features**, and
2. **Encoding categorical variables** appropriately.

### Dropping `entity_name`

- The `entity_name` is just an ID — it holds **no predictive value**.
- Including it can **lead to overfitting** or misleading encodings.
- We will safely **drop it**.

### Keeping `time_index` (optional but usually harmless)

- While `time_index` usually isn't informative in stationary time series,
- It **can be useful** if there are **regime shifts**, seasonal effects, or slow trends over time.
- Since it's **numeric and stable**, we choose to **keep it**.
- In the worst case, it’ll just be ignored by the model.

### Encoding `entity_class` and `property_class`

- These are **categorical variables** with meaningful distinctions.
- We will apply **one-hot encoding** to:
  - `entity_class` (e.g., Class_1, ..., Class_10)
  - `property_class` (e.g., Prop_1, ..., Prop_5)
- This allows models to pick up non-linear or interaction effects properly.

In [ ]:
# Save metadata for merging later
meta_columns = ml_df_target_1[['entity_name', 'time_index']].copy()

# Drop 'entity_name', keep 'time_index'
ml_df_target_1_encoded = ml_df_target_1.drop(columns=['entity_name'])

# One-hot encode categorical features
ml_df_target_1_encoded = pd.get_dummies(
    ml_df_target_1_encoded,
    columns=['entity_class', 'property_class'],
    drop_first=False
)

# Move target to the last column
cols = [col for col in ml_df_target_1_encoded.columns if col != 'target'] + ['target']
ml_df_target_1_encoded = ml_df_target_1_encoded[cols]

# Check result
ml_df_target_1_encoded

,time_index,size_lag_1,size_lag_2,size_lag_3,size_lag_4,size_lag_5,target_1_lag_1,target_1_lag_2,target_1_lag_3,target_1_lag_4,target_1_lag_5,entity_class_Class_1,entity_class_Class_3,entity_class_Class_7,property_class_Prop_1,property_class_Prop_2,property_class_Prop_3,property_class_Prop_4,property_class_Prop_5,target
0,21,1385.0,1379.0,1328.0,1230.0,1343.0,2208.0,2411.0,2352.0,2564.0,2625.0,True,False,False,False,False,True,False,False,2667.0
1,22,1620.0,1385.0,1379.0,1328.0,1230.0,2667.0,2208.0,2411.0,2352.0,2564.0,True,False,False,False,False,True,False,False,3540.0
2,23,1958.0,1620.0,1385.0,1379.0,1328.0,3540.0,2667.0,2208.0,2411.0,2352.0,True,False,False,False,False,True,False,False,3147.0
3,24,2117.0,1958.0,1620.0,1385.0,1379.0,3147.0,3540.0,2667.0,2208.0,2411.0,True,False,False,False,False,True,False,False,3968.0
4,25,2584.0,2117.0,1958.0,1620.0,1385.0,3968.0,3147.0,3540.0,2667.0,2208.0,True,False,False,False,False,True,False,False,4649.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,50,731.0,694.0,600.0,520.0,501.0,1044.0,1033.0,898.3,818.3,752.9,False,True,False,False,False,False,False,True,924.4
3186,34,1006.0,980.0,917.0,796.0,727.0,1441.0,1687.0,1590.0,1417.0,1013.0,False,True,False,False,False,True,False,False,2225.0
3187,35,1333.0,1006.0,980.0,917.0,796.0,2225.0,1441.0,1687.0,1590.0,1417.0,False,True,False,False,False,True,False,False,2209.0
3188,36,1383.0,1333.0,1006.0,980.0,917.0,2209.0,2225.0,1441.0,1687.0,1590.0,False,True,False,False,False,True,False,False,2634.0


## Data splitting in time series: why it’s not trivial

In traditional machine learning tasks, we often **randomly split** the dataset into a training set and a test set.

But in **time series problems**, that approach can lead to serious **information leakage**.


### What’s the problem?

Time series data has **temporal ordering**, meaning each row represents a value at a specific time.

If we **randomly shuffle the rows**, it’s possible that:
- A data point from **time step `t+1`** ends up in the training set,
- While a data point from **time step `t`** is in the test set.

This violates the core principle of time-based prediction — that the **future should not be known when making predictions**.

### A Real example of leakage

Let’s say:

- `y_{t-1}` appears in the test set,
- But `y_t` appears in the training set (because of shuffling).

A machine learning model could accidentally **learn from the future**, which is **unrealistic and invalid**.

### Safer approach: respect time order

Instead, we will split the data **based on the time index**:
- The **training set** will consist of earlier time steps,
- The **test set** will consist of later time steps.

If we have an explicit **validation (evaluation) set**. it has to repect this order too, meaning it has to come after training and before test sets.

This way, all model learning is based on **past data only**.

> This is another reason why we have kept the `time_index` column.

### Cross-validation with time series

This time-aware splitting must also extend to **cross-validation**.

Typical random K-fold splitting again suffers from the same leakage risk.

Instead, we should ideally use:

- **TimeSeriesSplit** from `sklearn.model_selection`
- Or create **manual rolling window splits**

These approaches ensure that each validation fold respects the **temporal direction** of data.

### Note on this notebook

For simplicity and interpretability in this notebook:

- We will perform a **single train-test split** based on time,
- And note that if cross-validation is applied, it must use **time-aware strategies**.

We will point this out again during the model evaluation stage.

In [ ]:
# Sort by time_index just in case
ml_df_target_1_encoded = ml_df_target_1_encoded.sort_values(by='time_index')

# Decide split point (80% of time steps)
time_cutoff = int(ml_df_target_1_encoded['time_index'].quantile(0.8))

# Training data: time_index <= cutoff
train_df_1 = ml_df_target_1_encoded[ml_df_target_1_encoded['time_index'] <= time_cutoff]

# Testing data: time_index > cutoff
test_df_1 = ml_df_target_1_encoded[ml_df_target_1_encoded['time_index'] > time_cutoff]

# Match time-based split
meta_train = meta_columns.loc[train_df_1.index]
meta_test = meta_columns.loc[test_df_1.index]

# Separate features and target
X_train_1 = train_df_1.drop(columns=['target'])
y_train_1 = train_df_1['target']

X_test_1 = test_df_1.drop(columns=['target'])
y_test_1 = test_df_1['target']

# Summary
print(f"Training samples: {len(X_train_1)}")
print(f"Testing samples: {len(X_test_1)}")
print(f"Time cutoff: {time_cutoff}")

Training samples: 2573
Testing samples: 617
Time cutoff: 45


## Baseline Model: Random Forest Regressor

Now that we’ve prepared our dataset and split it properly in a time-aware fashion, we can train a **first-pass baseline model**.

We’ll use a **Random Forest Regressor** — a powerful, non-parametric model that:
- Handles nonlinearities,
- Is robust to irrelevant features,
- Requires little feature scaling.

We’ll evaluate the predictions of `target_1` using:

- **MAE (Mean Absolute Error)**
- **RMSE (Root Mean Squared Error)**
- **R² Score (Coefficient of Determination)**

This will serve as a benchmark for improving future models.

In [ ]:
# Train model
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_1.drop(columns=['time_index']), y_train_1)

# Predict
yhat_test_1 = rf.predict(X_test_1.drop(columns=['time_index']))

# Evaluation
mae = mean_absolute_error(y_test_1, yhat_test_1)
rmse = np.sqrt(mean_squared_error(y_test_1, yhat_test_1))
r2 = r2_score(y_test_1, yhat_test_1)

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R² score: {r2:.3f}")

MAE: 16674.861
RMSE: 136190.201
R² score: 0.762


In [ ]:
# Merge metadata with predictions
results_df_1 = meta_test.copy()
results_df_1['true'] = y_test_1.values
results_df_1['predicted'] = yhat_test_1

# Pick an entity to visualize
selected_entity = results_df_1['entity_name'].unique()[0]  # You can change this

# Filter for this entity
single_entity_df = results_df_1[results_df_1['entity_name'] == selected_entity].copy()

# Melt for line plotting
melted = single_entity_df.melt(id_vars='time_index',
                               value_vars=['true', 'predicted'],
                               var_name='type', value_name='value')

# Plot
fig = px.line(
    melted,
    x='time_index',
    y='value',
    color='type',
    title=f'Predicted vs actual for entity: {selected_entity}'
)
fig.update_layout(width=900, height=400)
fig.show()

## Model selection and hyperparameter tuning

Now that we’ve trained a baseline model and evaluated it, the next step is to:

1. **Compare different machine learning models**
2. **Tune their hyperparameters**
3. **Select the best-performing model** based on time-aware cross-validation

### Why do this?

The performance of ML models heavily depends on:

- The **choice of model** (e.g., Random Forest, Gradient Boosting),
- And the **choice of hyperparameters** (e.g., number of trees, max depth, learning rate).

To make fair comparisons and avoid information leakage, we’ll use:

- **TimeSeriesSplit** for temporal cross-validation,
- **GridSearchCV** for tuning hyperparameters.

Our evaluation metric will be:

- **Mean Absolute Error (MAE)** — interpretable and robust to outliers

### Time-aware cross-validation

As we prepare to compare and tune our models, it’s critical to ensure that we evaluate them fairly and realistically.

In time series tasks, we must **preserve the temporal structure of the data**.

#### Why standard CV doesn't work

If we use regular k-fold cross-validation:
- It may shuffle time steps randomly,
- Which causes **information leakage**: the model can accidentally train on future data to predict the past.

This leads to **unrealistically optimistic results** — and models that don’t generalize well to real-time predictions.

#### Our tools

To solve this, we’ll use:
- `TimeSeriesSplit` from `sklearn.model_selection` to create **time-ordered train/test splits**
- `GridSearchCV` to run **exhaustive search over hyperparameter combinations**
- **Negative Mean Absolute Error (`neg_mean_absolute_error`)** as our scoring metric

#### Time-aware splitting review

We already introduced this earlier, but let’s re-emphasize it here for clarity:

> Time series data has **temporal ordering**, meaning each row represents a value at a specific time.
>
> If we randomly shuffle the rows, it’s possible that:
> - A data point from **time step `t+1`** ends up in the training set,
> - While a data point from **time step `t`** is in the test set.
>
> This violates the core principle of time-based prediction — that the **future should not be known when making predictions**.

This is why we’ll now use **`TimeSeriesSplit`** to evaluate all three models — `LinearRegression`, `RandomForestRegressor`, and `GradientBoostingRegressor` — in a consistent and safe way.

In [ ]:
# Set up time-aware cross-validator
tscv = TimeSeriesSplit(n_splits=5)

# Scoring metric: negative MAE (lower is better)
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Define models and param grids
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  # No hyperparameters to tune
    },
    'RandomForest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(),
        'params': {
            'n_estimators': [50, 100],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
    }
}

# Run grid search for each model
cv_results_1 = []

for name, config in models.items():
    print(f"Tuning: {name}")
    grid = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=tscv,
        scoring=scorer,
        n_jobs=-1
    )
    grid.fit(X_train_1.drop(columns=['time_index']), y_train_1)

    best_model = grid.best_estimator_
    best_score = -grid.best_score_  # negate because we used neg_MAE
    best_params = grid.best_params_

    cv_results_1.append({
        'model': name,
        'best_score_mae': best_score,
        'best_params': best_params,
        'estimator': best_model
    })

# Display summary
summary_df = pd.DataFrame(cv_results_1).drop(columns=['estimator'])
summary_df

Tuning: LinearRegression
Tuning: RandomForest
Tuning: GradientBoosting


,model,best_score_mae,best_params
0,LinearRegression,2911.760989,{}
1,RandomForest,4189.298145,"{'max_depth': 10, 'n_estimators': 50}"
2,GradientBoosting,4039.670414,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti..."


## The best model

Now, we can train the best model on full train set & predict on test set:

In [ ]:
# Retrieve best model (assuming lowest MAE)
best_result_1 = min(cv_results_1, key=lambda x: x['best_score_mae'])
best_model_name_1 = best_result_1['model']
best_model_1 = best_result_1['estimator']

print(f"Using best model: {best_model_name_1}")

# Predict on test data
yhat_test_1 = best_model_1.predict(X_test_1.drop(columns=['time_index']))

# Evaluation metrics
mae = mean_absolute_error(y_test_1, yhat_test_1)
rmse = np.sqrt(mean_squared_error(y_test_1, yhat_test_1))
r2 = r2_score(y_test_1, yhat_test_1)

print(f"MAE:  {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²:   {r2:.3f}")

Using best model: LinearRegression
MAE:  7439.288
RMSE: 34014.613
R²:   0.985


Let's visualize the results:

In [ ]:
# Create result dataframe
results_df_1 = meta_test.copy()  # from earlier — entity_name + time_index
results_df_1['true'] = y_test_1.values
results_df_1['predicted'] = yhat_test_1


single_entity_df_1 = results_df_1[results_df_1['entity_name'] == selected_entity].copy()

# Melt for plotting
melted = single_entity_df_1.melt(id_vars='time_index',
                                 value_vars=['true', 'predicted'],
                                 var_name='type', value_name='value')

# Plot
fig = px.line(
    melted,
    x='time_index',
    y='value',
    color='type',
    title=f'Predicted vs actual for target 1 — entity: {selected_entity}'
)
fig.update_layout(width=900, height=400)
fig.show()

## Modeling and predicting `target_2`

We’ve completed model selection, hyperparameter tuning, and evaluation for `target_1`.

Now, we turn our attention to **predicting `target_2`**, which is the other key variable in our final objective — computing and analyzing the **ratio `target_1 / target_2`**.

### What stays the same?

Since we already:
- Built lagged features for `target_2`,
- Encoded categorical variables the same way,
- Used the same time-based split cutoff (`time_cutoff`),
- And already set up cross-validation logic (`TimeSeriesSplit`, `GridSearchCV`),

...we can **reuse the same setup** for `target_2` with minimal adjustments.

### What changes?

We only need to:
1. Use the `ml_df_target_2` dataset (already constructed),
2. Extract the corresponding `target_2` column,
3. Split by time **in the same way**,
4. Run the same cross-validation and model selection logic,
5. Evaluate and visualize results.

Let’s proceed!

In [ ]:
# Drop 'entity_name', keep 'time_index'
ml_df_target_2_encoded = ml_df_target_2.drop(columns=['entity_name'])

# One-hot encode
ml_df_target_2_encoded = pd.get_dummies(
    ml_df_target_2_encoded,
    columns=['entity_class', 'property_class'],
    drop_first=False
)

# Move target to the end
cols = [col for col in ml_df_target_2_encoded.columns if col != 'target'] + ['target']
ml_df_target_2_encoded = ml_df_target_2_encoded[cols]

In [ ]:
# Sort by time and split using existing time_cutoff
ml_df_target_2_encoded = ml_df_target_2_encoded.sort_values(by='time_index')

train_df_2 = ml_df_target_2_encoded[ml_df_target_2_encoded['time_index'] <= time_cutoff]
test_df_2 = ml_df_target_2_encoded[ml_df_target_2_encoded['time_index'] > time_cutoff]

# Features and labels
X_train_2 = train_df_2.drop(columns=['target'])
y_train_2 = train_df_2['target']
X_test_2 = test_df_2.drop(columns=['target'])
y_test_2 = test_df_2['target']

In [ ]:
# Rerun model selection on target_2
cv_results_2 = []

for name, config in models.items():
    print(f"Tuning for target_2: {name}")
    grid = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=tscv,
        scoring=scorer,
        n_jobs=-1
    )
    grid.fit(X_train_2.drop(columns=['time_index']), y_train_2)

    best_model = grid.best_estimator_
    best_score = -grid.best_score_
    best_params = grid.best_params_

    cv_results_2.append({
        'model': name,
        'best_score_mae': best_score,
        'best_params': best_params,
        'estimator': best_model
    })

# Show comparison
pd.DataFrame(cv_results_2).drop(columns='estimator')

Tuning for target_2: LinearRegression
Tuning for target_2: RandomForest
Tuning for target_2: GradientBoosting


,model,best_score_mae,best_params
0,LinearRegression,3714.300918,{}
1,RandomForest,4526.613028,"{'max_depth': None, 'n_estimators': 100}"
2,GradientBoosting,4656.819476,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti..."


In [ ]:
# Pick best model
best_result_2 = min(cv_results_2, key=lambda x: x['best_score_mae'])
best_model_name_2 = best_result_2['model']
best_model_2 = best_result_2['estimator']

print(f"Best model for target_2: {best_model_name_2}")

# Predict
yhat_test_2 = best_model_2.predict(X_test_2.drop(columns=['time_index']))

# Evaluate
mae = mean_absolute_error(y_test_2, yhat_test_2)
rmse = np.sqrt(mean_squared_error(y_test_2, yhat_test_2))
r2 = r2_score(y_test_2, yhat_test_2)

print(f"MAE:  {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²:   {r2:.3f}")

Best model for target_2: LinearRegression
MAE:  10745.331
RMSE: 70300.508
R²:   0.952


In [ ]:
# Create result DataFrame
results_df_2 = meta_test.copy()
results_df_2['true'] = y_test_2.values
results_df_2['predicted'] = yhat_test_2

# Plot one entity
single_entity_df_2 = results_df_2[results_df_2['entity_name'] == selected_entity]

melted = single_entity_df_2.melt(id_vars='time_index',
                                 value_vars=['true', 'predicted'],
                                 var_name='type', value_name='value')

fig = px.line(
    melted,
    x='time_index',
    y='value',
    color='type',
    title=f'Predicted vs actual for target 2 — entity: {selected_entity}'
)
fig.update_layout(width=900, height=400)
fig.show()

## Combining predictions and computing ratios

We’ve trained models to independently predict both `target_1` and `target_2`.

Now, we combine their predictions to compute the **estimated ratio**:

$$
\widehat{\text{ratio}} = \frac{\widehat{\text{target}_1}}{\widehat{\text{target}_2}}
$$

This ratio is the **key metric** we want to analyze:
> _Which entity class, on average, produces the lowest ratio of `target_1` to `target_2`?_

Let’s compute the ratio and aggregate it by entity class.

In [ ]:
# Merge predictions from both targets
final_df = results_df_1[['entity_name', 'time_index', 'predicted']].rename(columns={'predicted': 'target_1_pred'})
final_df = final_df.merge(
    results_df_2[['entity_name', 'time_index', 'predicted']],
    on=['entity_name', 'time_index'],
    how='inner'
).rename(columns={'predicted': 'target_2_pred'})

# Merge entity class info
final_df = final_df.merge(
    ml_df[['entity_name', 'entity_class']].drop_duplicates(),
    on='entity_name',
    how='left'
)

# Compute predicted ratio
final_df['predicted_ratio'] = final_df['target_1_pred'] / final_df['target_2_pred']

In [ ]:
# Average predicted ratio per entity class
class_ratios = final_df.groupby('entity_class')['predicted_ratio'].mean().sort_values()

# Display
class_ratios_df = class_ratios.reset_index().rename(columns={'predicted_ratio': 'avg_predicted_ratio'})
class_ratios_df

,entity_class,avg_predicted_ratio
0,Class_7,-0.059337
1,Class_1,0.852555
2,Class_3,1.682348


In [ ]:
fig = px.bar(
    class_ratios_df,
    x='entity_class',
    y='avg_predicted_ratio',
    title='Average predicted ratio by entity class',
    text_auto='.2f'
)
fig.update_layout(xaxis_title='Entity class', yaxis_title='Avg predicted ratio')
fig.show()

## Class-level analysis of predicted ratios

Now that we have predicted both `target_1` and `target_2` and computed their ratio, we can analyze:

### Which **entity class** has the **lowest average** predicted ratio?

This might indicate:
- Higher efficiency (if `target_1` is something undesirable like disposal),
- More favorable behavior (e.g., higher `target_2` like recycling).

We'll compute the **mean ratio per class** and visualize the results.

In [ ]:
# Average predicted ratio per class
class_ratio_summary = (
    final_df
    .groupby('entity_class')['predicted_ratio']
    .mean()
    .sort_values()
    .reset_index()
    .rename(columns={'predicted_ratio': 'avg_predicted_ratio'})
)

# Display table
class_ratio_summary

,entity_class,avg_predicted_ratio
0,Class_7,-0.059337
1,Class_1,0.852555
2,Class_3,1.682348


In [ ]:
# Plot bar chart
fig = px.bar(
    class_ratio_summary,
    x='entity_class',
    y='avg_predicted_ratio',
    text_auto='.3f',
    title='Average predicted ratio per entity class',
    labels={'avg_predicted_ratio': 'Average Ratio'},
    color='avg_predicted_ratio',
    color_continuous_scale='Viridis'
)
fig.update_layout(xaxis_title='Entity class', yaxis_title='Avg predicted ratio', width=900)
fig.show()

## Predicting future

Now that we’ve trained models to predict `target_1` and `target_2`, we can also **predict for a future time index** — even one not present in the original dataset.

In this section, we’ll demonstrate how to predict the targets for **time index 51**.

### Why this is possible

To predict values at future time steps (like $t=51$), we need to **construct feature vectors** that mirror the ones used during training. This includes:

- ✅ Historical values for `target_1`, `target_2`, and `size` for lags $\{t-1, t-2, \dots, t-\tau\}$
- ✅ Entity class and property class (which are static per entity)
- ✅ Size lagged values only — **not** size at $t=51$
- ✅ **No leakage** from future targets or current-time exogenous variables

All this data **already exists** in our dataset up to $t=50$, so we can safely construct features for $t=51`.

Let’s build this future prediction set.

### Build future feature set for time index 51

In [ ]:
# Forecast target time
forecast_time = 51

# Use value kind 'A' (or any selected one)
df_vk = value_kind_datasets[selected_value_kind].copy()
df_vk = df_vk.sort_values(by=['entity_name', 'time_index'])
df_vk.set_index(['entity_name', 'time_index'], inplace=True)

# Containers for separate target lag structures
future_rows_1 = []
future_rows_2 = []

for entity in df_vk.index.get_level_values(0).unique():
    has_all_lags = True

    t1_lags = []
    t2_lags = []
    size_lags = []

    for lag in range(1, tau + 1):
        key = (entity, forecast_time - lag)
        if key in df_vk.index:
            past = df_vk.loc[key]
            t1_lags.append(past['target_1'])
            t2_lags.append(past['target_2'])
            size_lags.append(past['size'])
            entity_class = past['entity_class']
            property_class = past['property_class']
        else:
            has_all_lags = False
            break

    if not has_all_lags:
        continue

    row_base = {
        'entity_name': entity,
        'time_index': forecast_time,
        'entity_class': entity_class,
        'property_class': property_class,
        **{f'size_lag_{i+1}': size_lags[i] for i in range(tau)},
    }

    row_1 = {**row_base, **{f'target_1_lag_{i+1}': t1_lags[i] for i in range(tau)}}
    row_2 = {**row_base, **{f'target_2_lag_{i+1}': t2_lags[i] for i in range(tau)}}

    future_rows_1.append(row_1)
    future_rows_2.append(row_2)

# Create separate future DataFrames
future_df_1 = pd.DataFrame(future_rows_1)
future_df_2 = pd.DataFrame(future_rows_2)

### Prepare and predict

In [ ]:
future_meta = future_df_1[['entity_name', 'time_index']].copy()

# --- Future set for target_1 ---
future_encoded_1 = future_df_1.drop(columns=['entity_name'])

future_encoded_1 = pd.get_dummies(
    future_encoded_1,
    columns=['entity_class', 'property_class'],
    drop_first=False
)

# Align columns to training set
X_train_1_cols = set(X_train_1.columns)
missing_cols_1 = X_train_1_cols - set(future_encoded_1.columns)
for col in missing_cols_1:
    future_encoded_1[col] = 0
future_encoded_1 = future_encoded_1[X_train_1.columns]

# --- Future set for target_2 ---
future_encoded_2 = future_df_2.drop(columns=['entity_name'])

future_encoded_2 = pd.get_dummies(
    future_encoded_2,
    columns=['entity_class', 'property_class'],
    drop_first=False
)

# Align columns to training set
X_train_2_cols = set(X_train_2.columns)
missing_cols_2 = X_train_2_cols - set(future_encoded_2.columns)
for col in missing_cols_2:
    future_encoded_2[col] = 0
future_encoded_2 = future_encoded_2[X_train_2.columns]

### Predict targets and ratio for future

In [ ]:
# Predict
future_df_1['target_1_pred'] = best_model_1.predict(future_encoded_1.drop(columns=['time_index']))
future_df_2['target_2_pred'] = best_model_2.predict(future_encoded_2.drop(columns=['time_index']))

# Merge and compute ratio
pred_future = future_df_1[['entity_name', 'time_index', 'target_1_pred']].merge(
    future_df_2[['entity_name', 'target_2_pred']],
    on='entity_name'
)

# Add class info
pred_future = pred_future.merge(
    ml_df[['entity_name', 'entity_class']].drop_duplicates(),
    on='entity_name',
    how='left'
)

pred_future['predicted_ratio'] = pred_future['target_1_pred'] / pred_future['target_2_pred']

### Visualize future ratio by class

In [ ]:
# Summarize per class
future_summary = (
    pred_future.groupby('entity_class')['predicted_ratio']
    .mean()
    .sort_values()
    .reset_index()
    .rename(columns={'predicted_ratio': 'future_avg_predicted_ratio'})
)

# Plot
fig = px.bar(
    future_summary,
    x='entity_class',
    y='future_avg_predicted_ratio',
    title='Predicted ratios at time index 51 by entity class',
    text_auto='.3f',
    color='future_avg_predicted_ratio',
    color_continuous_scale='Plasma'
)
fig.update_layout(xaxis_title='Entity class', yaxis_title='Predicted ratio (t=51)', width=900)
fig.show()

## Predicting further into the future

So far, we’ve predicted the next time step (`t = 51`) using past values from `t = 50` and earlier. But what if we want to go **5 or 10 steps ahead**, say all the way to `t = 60`?

This task is called **multi-step forecasting**, and it's a common challenge in time series.

### What’s the challenge?

The further into the future you go:
- The **less historical information** you have available,
- The more you need to rely on **your own predictions** as inputs for future ones,
- The **more uncertain** your estimates become.

### Strategy 1: recursive forecasting (autoregressive rolling prediction)

Predict one step ahead, then **use that prediction** as an input to predict the next step:
1. Predict \( y_{t+1} \) using historical data,
2. Append \( y_{t+1} \) to the lag features and predict \( y_{t+2} \),
3. Repeat up to \( y_{t+h} \).

✅ Easy to implement.  
⚠️ Error accumulation can become significant.  
⚠️ **Important**: Since size is a time-varying exogenous input in our dataset, we must also:
   - Build a model to **predict size at future time steps**,
   - Use those predicted sizes to construct input features for each step.

### Strategy 2: Direct forecasting

Train **separate models** for each horizon:
- One model for \( y_{t+1} \),
- Another for \( y_{t+2} \), and so on.

✅ Avoids recursive error buildup.  
⚠️ Requires more data, more modeling, and careful tracking of model structure.


### Strategy 3: Multi-output models

Train a **single model** that outputs all future steps:
$$
[y_{t+1}, y_{t+2}, \dots, y_{t+h}] = f(x_{t}, x_{t-1}, \dots, x_{t-\tau})
$$

✅ Efficient and consistent.  
⚠️ Requires more complex models and carefully structured training data.


### For our case

If we wanted to predict all the way to `t = 60`, we could:

- Use the recursive method to build future lags dynamically,
- Build and use a **secondary model to predict size at future time steps**,
- Reuse the trained model to roll forward predictions,
- Track and store predicted values for both `target_1` and `target_2`,
- Then compute predicted ratios per step.

---

## Conclusion

In this notebook, we walked through the **end-to-end process** of solving a complex time series prediction task using machine learning. We:

- ✅ **Generated a rich synthetic dataset** with:
  - Multiple value kinds,
  - Entity classes and properties,
  - Time-varying and static features,
  - Nonlinear relationships and noise.

- ✅ **Explored and visualized** the dataset to understand trends and distributions.

- ✅ **Reframed the problem** into a machine learning task by:
  - Engineering lag-based features from historical target values,
  - Incorporating exogenous variables like size and class,
  - Being careful to respect the **temporal structure** of the data.

- ✅ **Trained and evaluated ML models** (Linear Regression, Random Forest, Gradient Boosting) with:
  - Time-aware train/test splits,
  - Grid search and cross-validation using `TimeSeriesSplit`.

- ✅ **Predicted target values**, computed their ratio, and identified the entity class with the **lowest average ratio** — our original goal.

- ✅ **Built future predictions** (e.g., for time index 51) using autoregressive logic and predicted size-aware input features.

- ✅ Discussed **strategies for long-horizon forecasting**, including recursive, direct, and multi-output approaches.


### What you’ve learned

This notebook was more than just a prediction task — it was a practical deep dive into:

- How time series problems differ from standard ML,
- How data structure affects learning,
- How to carefully build robust, temporally aware ML pipelines.

You're now ready to tackle real-world forecasting challenges with insight and confidence.

### Next steps

Some ideas to extend this work:

- Train **neural time series models** like LSTMs or Transformers,
- Experiment with **missing data**, irregular intervals, or anomalies,
- Add domain-specific features (seasonality, policy changes),
- Explore **probabilistic models** or **uncertainty quantification**.